
### Exploration of the boston_2016 bluebikes dataset for cleaning preparation


In [1]:
# from WS_05

# import all relevant libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import seaborn as sns
sns.set()
sns.set_style("white")
sns.set_palette("GnBu_d")

import folium
from folium import plugins
from folium.plugins import HeatMap

import math
import random

from datetime import date, time, datetime, timedelta

In [2]:
### read in Boston Bike data to a Dataframe

boston = pd.read_csv("boston_2016.csv")

In [3]:
boston.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1236203 entries, 0 to 1236202
Data columns (total 8 columns):
 #   Column              Non-Null Count    Dtype 
---  ------              --------------    ----- 
 0   start_time          1236203 non-null  object
 1   end_time            1236203 non-null  object
 2   start_station_id    1236203 non-null  int64 
 3   end_station_id      1236203 non-null  int64 
 4   start_station_name  1236203 non-null  object
 5   end_station_name    1236203 non-null  object
 6   bike_id             1236203 non-null  int64 
 7   user_type           1236203 non-null  object
dtypes: int64(3), object(5)
memory usage: 75.5+ MB


### 0 null values in the dataset

In [4]:
### change boston start_time and end_time from object to datetime, remove microseconds

boston["start_time"] = pd.to_datetime(boston["start_time"])
boston["start_time"] = boston["start_time"].apply(lambda x: x.replace(microsecond = 0))

boston["end_time"] = pd.to_datetime(boston["end_time"])
boston["end_time"] = boston["end_time"].apply(lambda x: x.replace(microsecond = 0))
boston

,start_time,end_time,start_station_id,end_station_id,start_station_name,end_station_name,bike_id,user_type
0,2016-01-01 00:08:07,2016-01-01 00:40:26,36,19,Boston Public Library - 700 Boylston St.,Buswell St. at Park Dr.,1406,Subscriber
1,2016-01-01 00:08:19,2016-01-01 00:40:27,36,19,Boston Public Library - 700 Boylston St.,Buswell St. at Park Dr.,1121,Subscriber
2,2016-01-01 00:10:15,2016-01-01 00:40:29,36,19,Boston Public Library - 700 Boylston St.,Buswell St. at Park Dr.,474,Subscriber
3,2016-01-01 00:10:22,2016-01-01 00:40:11,36,19,Boston Public Library - 700 Boylston St.,Buswell St. at Park Dr.,1504,Subscriber
4,2016-01-01 00:15:36,2016-01-01 00:31:17,36,67,Boston Public Library - 700 Boylston St.,MIT at Mass Ave / Amherst St,913,Customer
...,...,...,...,...,...,...,...,...
1236198,2016-12-31 23:01:46,2016-12-31 23:14:20,8,33,Union Square - Brighton Ave at Cambridge St,Kenmore Square,230,Subscriber
1236199,2016-12-31 23:14:21,2016-12-31 23:18:03,190,6,Nashua Street at Red Auerbach Way,Cambridge St at Joy St,1420,Subscriber
1236200,2016-12-31 23:15:38,2016-12-31 23:24:43,33,36,Kenmore Square,Copley Square - Dartmouth St at Boylston St,230,Subscriber
1236201,2016-12-31 23:20:35,2016-12-31 23:27:58,90,141,Lechmere Station at Cambridge St / First St,Kendall Street,74,Subscriber


In [5]:
boston.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1236203 entries, 0 to 1236202
Data columns (total 8 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   start_time          1236203 non-null  datetime64[ns]
 1   end_time            1236203 non-null  datetime64[ns]
 2   start_station_id    1236203 non-null  int64         
 3   end_station_id      1236203 non-null  int64         
 4   start_station_name  1236203 non-null  object        
 5   end_station_name    1236203 non-null  object        
 6   bike_id             1236203 non-null  int64         
 7   user_type           1236203 non-null  object        
dtypes: datetime64[ns](2), int64(3), object(3)
memory usage: 75.5+ MB


### Data is sortet by start_time

In [6]:
### add the trip duration as timedelta for each tour

boston["trip_duration"] = boston["end_time"] - boston["start_time"]
boston

,start_time,end_time,start_station_id,end_station_id,start_station_name,end_station_name,bike_id,user_type,trip_duration
0,2016-01-01 00:08:07,2016-01-01 00:40:26,36,19,Boston Public Library - 700 Boylston St.,Buswell St. at Park Dr.,1406,Subscriber,0 days 00:32:19
1,2016-01-01 00:08:19,2016-01-01 00:40:27,36,19,Boston Public Library - 700 Boylston St.,Buswell St. at Park Dr.,1121,Subscriber,0 days 00:32:08
2,2016-01-01 00:10:15,2016-01-01 00:40:29,36,19,Boston Public Library - 700 Boylston St.,Buswell St. at Park Dr.,474,Subscriber,0 days 00:30:14
3,2016-01-01 00:10:22,2016-01-01 00:40:11,36,19,Boston Public Library - 700 Boylston St.,Buswell St. at Park Dr.,1504,Subscriber,0 days 00:29:49
4,2016-01-01 00:15:36,2016-01-01 00:31:17,36,67,Boston Public Library - 700 Boylston St.,MIT at Mass Ave / Amherst St,913,Customer,0 days 00:15:41
...,...,...,...,...,...,...,...,...,...
1236198,2016-12-31 23:01:46,2016-12-31 23:14:20,8,33,Union Square - Brighton Ave at Cambridge St,Kenmore Square,230,Subscriber,0 days 00:12:34
1236199,2016-12-31 23:14:21,2016-12-31 23:18:03,190,6,Nashua Street at Red Auerbach Way,Cambridge St at Joy St,1420,Subscriber,0 days 00:03:42
1236200,2016-12-31 23:15:38,2016-12-31 23:24:43,33,36,Kenmore Square,Copley Square - Dartmouth St at Boylston St,230,Subscriber,0 days 00:09:05
1236201,2016-12-31 23:20:35,2016-12-31 23:27:58,90,141,Lechmere Station at Cambridge St / First St,Kendall Street,74,Subscriber,0 days 00:07:23


In [7]:
boston.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1236203 entries, 0 to 1236202
Data columns (total 9 columns):
 #   Column              Non-Null Count    Dtype          
---  ------              --------------    -----          
 0   start_time          1236203 non-null  datetime64[ns] 
 1   end_time            1236203 non-null  datetime64[ns] 
 2   start_station_id    1236203 non-null  int64          
 3   end_station_id      1236203 non-null  int64          
 4   start_station_name  1236203 non-null  object         
 5   end_station_name    1236203 non-null  object         
 6   bike_id             1236203 non-null  int64          
 7   user_type           1236203 non-null  object         
 8   trip_duration       1236203 non-null  timedelta64[ns]
dtypes: datetime64[ns](2), int64(3), object(3), timedelta64[ns](1)
memory usage: 84.9+ MB


### Gather information

In [8]:
boston.nunique()

start_time            1176852
end_time              1175109
start_station_id          189
end_station_id            189
start_station_name        276
end_station_name          276
bike_id                  1802
user_type                   2
trip_duration           14031
dtype: int64

### 189 distinct start and end station_ids, but 276 distinct start and end station_names 
### 1802 bike ids
### 2 User types

In [9]:
boston.groupby('start_station_id').start_station_name.nunique()

start_station_id
1      1
3      2
4      2
5      2
6      2
      ..
215    2
216    1
217    1
218    2
219    2
Name: start_station_name, Length: 189, dtype: int64

In [10]:
boston.groupby('start_station_id').start_station_name.unique()

start_station_id
1                                [18 Dorrance Warehouse]
3      [Colleges of the Fenway, Colleges of the Fenwa...
4      [Tremont St. at Berkeley St., Tremont St at E ...
5      [Northeastern U / North Parking Lot, Northeast...
6      [Cambridge St. at Joy St., Cambridge St at Joy...
                             ...                        
215    [The Eddy at New Street, The Eddy - New St at ...
216                            [Glendon St at Condor St]
217    [Orient Heights T Stop - Bennington St at Sara...
218    [Watermark Seaport, Watermark Seaport - Boston...
219    [Central Square East Boston - Porter Street at...
Name: start_station_name, Length: 189, dtype: object

### Some station ids have 2 station names assigned, possibly because they were renamed
### for example The Eddy at New Street and The Eddy - New St at Sumemer St with station_id 215

### trip duration analysis

In [11]:
print(boston["trip_duration"].max())

346 days 03:14:39


In [12]:
print(boston["trip_duration"].min())

-1 days +23:06:24


In [13]:
print(boston["trip_duration"].mean())

0 days 00:22:45.223969687


### max trip duration almost a year and min duration negative, cleaning needed

https://help.bluebikes.com/hc/en-us/articles/360034926492-How-long-can-I-keep-a-bike-out-
https://help.bluebikes.com/hc/en-us/articles/360034926452-What-if-I-keep-a-bike-out-too-long-

### longest pass allows 2 hours, every 30 minutes cost extra 2,5 or 2$ (Income eligible members)
### single rides 30 min
### monthly and anual membership 45 min 
### adventure pass up to 2 hours
### Income eligible members (qualified for a discounted monthly or annual memberships) 60 min

### if the bike is not returned after 24 hours, a stolen/lost bike fee will be charged of 1200$ + tax



In [14]:
### define some timedeltas for trip_duration check

### time_delta for 0 Minutes
date1 = pd.Timedelta(pd.offsets.Minute(0))

### time_delta for 2 Hours
date2 = pd.Timedelta(pd.offsets.Hour(2))

### time_delta for 1 Day
date3 = pd.Timedelta(pd.offsets.Day(1))


In [15]:
### nb. of trips over 24 hours
len(boston.loc[boston.trip_duration > date3])

1079

In [20]:
### nb. of trips with negaive time
len(boston.loc[boston.trip_duration < date1])

3

In [16]:
### nb. of trips over 2 hours 
len(boston.loc[boston.trip_duration > date2])

8627

In [17]:
### nb. of trips over 2 hours and under 24 hours
print((len(boston.loc[boston.trip_duration > date2]) - len(boston.loc[boston.trip_duration > date3]) ))

7548


### remove negative trip duration and the trips over 24 hours

In [28]:
bostonC = boston[(boston["trip_duration"] > date1) & (boston["trip_duration"] < date3)]
bostonC.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1235121 entries, 0 to 1236202
Data columns (total 9 columns):
 #   Column              Non-Null Count    Dtype          
---  ------              --------------    -----          
 0   start_time          1235121 non-null  datetime64[ns] 
 1   end_time            1235121 non-null  datetime64[ns] 
 2   start_station_id    1235121 non-null  int64          
 3   end_station_id      1235121 non-null  int64          
 4   start_station_name  1235121 non-null  object         
 5   end_station_name    1235121 non-null  object         
 6   bike_id             1235121 non-null  int64          
 7   user_type           1235121 non-null  object         
 8   trip_duration       1235121 non-null  timedelta64[ns]
dtypes: datetime64[ns](2), int64(3), object(3), timedelta64[ns](1)
memory usage: 94.2+ MB


In [29]:
print(bostonC["trip_duration"].max())

0 days 23:56:42


In [30]:
print(bostonC["trip_duration"].min())

0 days 00:01:01


In [31]:
print(bostonC["trip_duration"].mean())

0 days 00:15:55.161627079
